In [15]:
import xarray as xr
import os
import argparse
import pickle
import numpy as np
import experiments as exp
import VED
import tensorflow as tf
import preprocessing
import standardize
import importlib as imp

In [16]:
EXPERIMENT = 'test_exp'

# -------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------

# get experiment settings
# assert np.isin(ARGS.exp_name, exp.experiments.keys()), "no experiment with the name" + ARGS.exp_name + ", current experiments:" + exp.experiments.keys()
experiment_settings = exp.get_experiment(EXPERIMENT)

# -------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------

# set up random seed
tf.keras.backend.clear_session()
np.random.seed(experiment_settings["seed"])
# random.seed(experiment_settings["seed"])
tf.random.set_seed(experiment_settings["seed"])

In [3]:
##### pre-process data
# get random indices for training and validation
# rng_seed = np.random.default_rng(experiment_settings["seed"])
# total_members = np.arange(np.sum(experiment_settings["nmembers"])) # assuming this input is a list of train, val members
# rnd_inds = []
# for nmem in experiment_settings["nmembers"]:
#     rnd_inds.append(rng_seed.choice(total_members, nmem, replace=False))
#     total_members = np.setdiff1d(total_members, rnd_inds)

for ii, var in enumerate(experiment_settings["input_variable"]):
    # load the training and validation sets for this variable
    At, Ft, It = preprocessing.make_data(models=experiment_settings["train_models"], var=var,
                                      timecut=experiment_settings["time_range"], mems=experiment_settings["train_members"])

    Av, Fv, Iv = preprocessing.make_data(models=experiment_settings["val_models"], var=var,
                                      timecut=experiment_settings["time_range"], mems=experiment_settings["val_members"])
    
    
    # put these into an array in the shape [samples, lat, lon, variable]
    if ii == 0:
        Atrain = At[:, :, :, np.newaxis]
        Aval = Av[:, :, :, np.newaxis]
        Ftrain = Ft[:, :, :, np.newaxis]
        Fval = Fv[:, :, :, np.newaxis]
        Itrain = It[:, :, :, np.newaxis]
        Ival = Iv[:, :, :, np.newaxis]
    else:
        Atrain = np.concatenate([Atrain, At[:, :, :, np.newaxis]], axis=-1)
        Aval = np.concatenate([Aval, Av[:, :, :, np.newaxis]], axis=-1)
        Ftrain = np.concatenate([Ftrain, Ft[:, :, :, np.newaxis]], axis=-1)
        Fval = np.concatenate([Fval, Fv[:, :, :, np.newaxis]], axis=-1)
        Itrain = np.concatenate([Itrain, It[:, :, :, np.newaxis]], axis=-1)
        Ival = np.concatenate([Ival, Iv[:, :, :, np.newaxis]], axis=-1)
        
    if var == experiment_settings["target_variable"]:
        target_ind = ii
        
# get the target variable
Ftrain = Ftrain[:, :, :, target_ind]
Itrain = Itrain[:, :, :, target_ind]
    
# -------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------

# get the correct evaluation data to predict
Atest = preprocessing.make_eval_mem(evalmem="1H",var=experiment_settings["target_variable"],timecut="Tier1")

/glade/campaign/cgd/cas/asphilli/ForceSMIP/ Omon tos CESM2
/glade/campaign/cgd/cas/asphilli/ForceSMIP/ Omon tos CESM2
/glade/campaign/cgd/cas/asphilli/ForceSMIP/ Amon pr CESM2
/glade/campaign/cgd/cas/asphilli/ForceSMIP/ Amon pr CESM2


In [17]:
imp.reload(exp)
imp.reload(standardize)

<module 'standardize' from '/glade/u/home/egordon/ForceSMIP/standardize.py'>

In [14]:
# standardize and select 'internal' or 'forced'
# jamin's datadoer here
Doer = standardize.DataDoer(Atrain, Aval, Atest, Ftrain, Fval, Itrain, Ival, experiment_settings)
Xtrain, Xval, Xtest, Ttrain, Tval = Doer.do()

Returning (Xtrain, Xval, Xtest, Ttrain, Tval)


NameError: name 'Ftrain' is not defined

In [ ]:
# build model
ved, encoder, decoder = VED.build_VED(Xtrain, experiment_settings)

In [ ]:
# train model
ved, encoder, decoder, history = VED.train_VED(Xtrain, Ttrain, Xval, Tval, experiment_settings)

In [ ]:
# save trained model
tf.keras.models.save_model(ved, os.path.join('saved_models', exp_name, exp_name+str(experiment_settings["seed"])+"_model"), overwrite=False)
ved.save_weights(os.path.join('saved_models', exp_name, exp_name+str(experiment_settings["seed"])+"_weights.h5"))
with open(os.path.join('saved_models', exp_name, exp_name+str(experiment_settings["seed"])+"_history.pickle"), "wb") as handle:
        pickle.dump(ved.history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# make predictions and save standardized and unstandardized
Ptrain = ved.predict(Xtrain)
Pval = ved.predict(Xval)
Ptest = ved.predict(Xtest)

Ptrain_us, Pval_us, Ptest_us = Doer.unstandardize(Ptrain, Pval, Ptest)

Ptrain_out = Atrain-Ptrain_us
Pval_out = Aval-Pval_us
Ptest_out = Atest-Ptest_us

arr_name = 'saved_predictions/' + exp_name+str(experiment_settings["seed"])+"_preds.npz"
np.savez(arr_name,Ptrain=Ptrain_out,
                  Pval=Pval_out,
                  Ptest=Ptest_out,
                  )